## 문제 풀이

### [문제105] yob2013 ~ yob2016 년도별 출생수를 출력해주세요.

In [16]:
import pandas as pd 

# for문 활용 

years = []

for i in range(2013,2017):
    df = pd.read_csv("c:/data/yob{}.txt".format(i), names = ['name','gender','birth'])
    years.append([i,df['birth'].sum()])

years

[[2013, 3634744], [2014, 3692930], [2015, 3683749], [2016, 3637321]]

In [17]:
# 연도별 birth.sum()을 txt로 저장하기

with open("c:/data/years.txt","w") as file:
    for i, j in years:
        data = '{},{}\n'.format(i,j)
        file.write(data)

### [문제106] yob2013 ~ yob2016 성별 출생 현황을 year_gender_total.csv 파일로 생성해주세요.

In [18]:
# solution 1 : file에 틀을 만들고 시작

file = open("c:/data/yaer_gender_total.csv","w")
file.write("년도,남자,여자\n")

for i in range(2013,2017):
    temp = pd.read_csv("c:/data/yob{}.txt".format(i), names = ['name','gender','birth'])
    m = temp[temp['gender'] == 'M']['birth'].sum()
    f = temp[temp['gender'] == 'F']['birth'].sum()
    txt = '{},{}.{}\n'.format(i,m,f)
    file.write(txt)

file.close()

In [20]:
# solution 2 : csv file로 직접 쓰는 방법 

import csv 

# newline : 빈 공백줄 없애기

with open("c:/data/year_gender_total_new.csv","w",newline="" ) as file:
    w = csv.writer(file)
    w.writerow(['년도','여자','남자']) # 행 추가 
    
    for i in range(2013,2017):
        temp = pd.read_csv('c:/data/yob{}.txt'.format(i),names=['name','gender','birth'])
        x = temp['birth'].groupby(temp['gender']).sum() #collections.Counter 한거랑 같겠다
        w.writerow([i,x.loc['F'],x.loc['M']]) # 차례대로 행 추가하기 

In [25]:
# solution 3 : df로 만들기 
## ignore_index = True, 안하면 오류가 뜹니다 
### 기존 df가 가지고 있는 index, 새로운 df가 가지고 있는 index가 충돌하기 때문에 기존 df가 가지고 있는 index를 무시하자 라는 명령을 넣어야 한다
## x.to_csv : 데이터 프레임을 csv 파일로 떨어트려야 할때

from pandas import Series, DataFrame 

new= DataFrame() # 빈 df 만들기 가능 
new

df = DataFrame(columns = ['년도','여자','남자'])
df

for i in range(2013,2017):
    temp = pd.read_csv('c:/data/yob{}.txt'.format(i),names = ['name','gender','birth'])
    x = temp['birth'].groupby(temp['gender']).sum()
    df = df.append({'년도':i,'여자':x.loc['F'], '남자':x.loc['M']},ignore_index=True)     

C:\Users\82108\AppData\Local\Temp\ipykernel_13564\1133679798.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'년도':i,'여자':x.loc['F'], '남자':x.loc['M']},ignore_index=True)
C:\Users\82108\AppData\Local\Temp\ipykernel_13564\1133679798.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'년도':i,'여자':x.loc['F'], '남자':x.loc['M']},ignore_index=True)
C:\Users\82108\AppData\Local\Temp\ipykernel_13564\1133679798.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'년도':i,'여자':x.loc['F'], '남자':x.loc['M']},ignore_index=True)
C:\Users\82108\AppData\Local\Temp\ipykernel_13564\1133679798.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futur

In [26]:
## ★to_csv: pandas에서  제공, df를 csv 파일로 떨어트려야 할 때 (더 쉬움)

df.to_csv("c:/data/new.csv",index=False)

**file을 열때 주의**

없는 파일까지 포함해서 열려고 하면

*FileNotFoundError: [Errno 2] No such file or directory: 'c:/data/yob1995.txt'*
이런 에러가 뜬다.

### [문제 108] 자신의 부서 평균 급여보다 더 많이 받는 사원들의 EMPLOYEE_ID, SALARY, AVG_SAL, DEPARTMENT_ID, DEPARTMENT_NAME 정보를 파일로 저장해주세요 

In [31]:
# 데이터 불러오기

emp = pd.read_csv("c:/data/employees.csv")
dept = pd.read_csv("c:/data/departments.csv")

In [33]:
# 데이터 확인하기
emp.head()

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0
1,101,Neena,Kochhar,NKOCHHAR,515.123.4568,2005-09-21,AD_VP,17000,NaN,100.0,90.0
2,102,Lex,De Haan,LDEHAAN,515.123.4569,2001-01-13,AD_VP,17000,NaN,100.0,90.0
3,103,Alexander,Hunold,AHUNOLD,590.423.4567,2006-01-03,IT_PROG,9000,NaN,102.0,60.0
4,104,Bruce,Ernst,BERNST,590.423.4568,2007-05-21,IT_PROG,6000,NaN,103.0,60.0


In [34]:
dept.head()

,DEPARTMENT_ID,DEPARTMENT_NAME,MANAGER_ID,LOCATION_ID
0,10,Administration,200.0,1700
1,20,Marketing,201.0,1800
2,30,Purchasing,114.0,1700
3,40,Human Resources,203.0,2400
4,50,Shipping,121.0,1500


In [36]:
# solution 1 

# step 1 : 부서별 평균을 만들기 

dept_sal = emp['SALARY'].groupby(emp['DEPARTMENT_ID']).mean()
dept_sal.astype(int)

DEPARTMENT_ID
10.0      4400
20.0      9500
30.0      4150
40.0      6500
50.0      3475
60.0      5760
70.0     10000
80.0      8955
90.0     19333
100.0     8601
110.0    10154
Name: SALARY, dtype: int32

In [38]:
# step 2 : avg_sal이랑 salary랑 비교하기 위해서 merge 시켜주기

y = pd.merge(dept_sal, emp, left_index = True, right_on = 'DEPARTMENT_ID')
y.info() # 똑같은 컬럼명이 있으면 x,y로 분리한다 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 100 to 106
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   SALARY_x        106 non-null    float64
 1   EMPLOYEE_ID     106 non-null    int64  
 2   FIRST_NAME      106 non-null    object 
 3   LAST_NAME       106 non-null    object 
 4   EMAIL           106 non-null    object 
 5   PHONE_NUMBER    106 non-null    object 
 6   HIRE_DATE       106 non-null    object 
 7   JOB_ID          106 non-null    object 
 8   SALARY_y        106 non-null    int64  
 9   COMMISSION_PCT  34 non-null     float64
 10  MANAGER_ID      105 non-null    float64
 11  DEPARTMENT_ID   106 non-null    float64
dtypes: float64(4), int64(2), object(6)
memory usage: 10.8+ KB


In [39]:
# SALARY 이름 변경

y = y.rename(columns = {'SALARY_x':'AVG_SAL'})
y = y.rename(columns = {'SALARY_y':'SALARY'})

y.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 106 entries, 100 to 106
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   AVG_SAL         106 non-null    float64
 1   EMPLOYEE_ID     106 non-null    int64  
 2   FIRST_NAME      106 non-null    object 
 3   LAST_NAME       106 non-null    object 
 4   EMAIL           106 non-null    object 
 5   PHONE_NUMBER    106 non-null    object 
 6   HIRE_DATE       106 non-null    object 
 7   JOB_ID          106 non-null    object 
 8   SALARY          106 non-null    int64  
 9   COMMISSION_PCT  34 non-null     float64
 10  MANAGER_ID      105 non-null    float64
 11  DEPARTMENT_ID   106 non-null    float64
dtypes: float64(4), int64(2), object(6)
memory usage: 10.8+ KB


In [40]:
# step 3 : avg_sal과 salary 비교하기 

yy = y[['EMPLOYEE_ID', 'DEPARTMENT_ID','SALARY','AVG_SAL']]
yy.head()

,EMPLOYEE_ID,DEPARTMENT_ID,SALARY,AVG_SAL
100,200,10.0,4400,4400.0
101,201,20.0,13000,9500.0
102,202,20.0,6000,9500.0
14,114,30.0,11000,4150.0
15,115,30.0,3100,4150.0


In [44]:
result = pd.merge(yy.query('SALARY > AVG_SAL'), dept[['DEPARTMENT_ID','DEPARTMENT_NAME']])

result.head()

,EMPLOYEE_ID,DEPARTMENT_ID,SALARY,AVG_SAL,DEPARTMENT_NAME
0,201,20.0,13000,9500.000000,Marketing
1,114,30.0,11000,4150.000000,Purchasing
2,120,50.0,8000,3475.555556,Shipping
3,121,50.0,8200,3475.555556,Shipping
4,122,50.0,7900,3475.555556,Shipping


In [ ]:
# step 4 : result 저장하기

result.to_csv("c:/data/dept_sal.csv",index=False,float_format='%.1f') # 소수점 첫째자리 까지 출력 
result.to_csv("c:/data/dept_sal.csv",index=False,float_format='%.f') # 정수로 저장

In [45]:
# soltuion 2 : 조금 더 간단하게 만들기, emp 테이블만 활용함 

new = DataFrame()

for n, g in emp[['EMPLOYEE_ID','DEPARTMENT_ID','SALARY']].groupby('DEPARTMENT_ID'):
    print("*"*20)
    print(n) # department_id
    print(g)
    print('부서평균',g.SALARY.mean())
    print(g[g.SALARY > g.SALARY.mean()])
    new = new.append(g[g.SALARY > g.SALARY.mean()])

    
new

********************
10.0
     EMPLOYEE_ID  DEPARTMENT_ID  SALARY
100          200           10.0    4400
부서평균 4400.0
Empty DataFrame
Columns: [EMPLOYEE_ID, DEPARTMENT_ID, SALARY]
Index: []
********************
20.0
     EMPLOYEE_ID  DEPARTMENT_ID  SALARY
101          201           20.0   13000
102          202           20.0    6000
부서평균 9500.0
     EMPLOYEE_ID  DEPARTMENT_ID  SALARY
101          201           20.0   13000
********************
30.0
    EMPLOYEE_ID  DEPARTMENT_ID  SALARY
14          114           30.0   11000
15          115           30.0    3100
16          116           30.0    2900
17          117           30.0    2800
18          118           30.0    2600
19          119           30.0    2500
부서평균 4150.0
    EMPLOYEE_ID  DEPARTMENT_ID  SALARY
14          114           30.0   11000
********************
40.0
     EMPLOYEE_ID  DEPARTMENT_ID  SALARY
103          203           40.0    6500
부서평균 6500.0
Empty DataFrame
Columns: [EMPLOYEE_ID, DEPARTMENT_ID, SALARY]
Ind

C:\Users\82108\AppData\Local\Temp\ipykernel_13564\1538227261.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new = new.append(g[g.SALARY > g.SALARY.mean()])
C:\Users\82108\AppData\Local\Temp\ipykernel_13564\1538227261.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new = new.append(g[g.SALARY > g.SALARY.mean()])
C:\Users\82108\AppData\Local\Temp\ipykernel_13564\1538227261.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new = new.append(g[g.SALARY > g.SALARY.mean()])
C:\Users\82108\AppData\Local\Temp\ipykernel_13564\1538227261.py:11: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new = new.append(g[g.SALARY > g.SALARY.mean(

,EMPLOYEE_ID,DEPARTMENT_ID,SALARY
101,201,20.0,13000
14,114,30.0,11000
20,120,50.0,8000
21,121,50.0,8200
22,122,50.0,7900
23,123,50.0,6500
24,124,50.0,5800
37,137,50.0,3600
41,141,50.0,3500
84,184,50.0,4200


In [47]:
## solution 3: concat

new = DataFrame() 

for n, g in emp[['EMPLOYEE_ID','SALARY','DEPARTMENT_ID']].groupby('DEPARTMENT_ID'):
    print("*" * 20)
    print(n) # department_id
    print(g) 
    print('부서평균: ', g.SALARY.mean())
    print(g[g.SALARY> g.SALARY.mean()])
    new = pd.concat([new,g[g.SALARY> g.SALARY.mean()]])

new


********************
10.0
     EMPLOYEE_ID  SALARY  DEPARTMENT_ID
100          200    4400           10.0
부서평균:  4400.0
Empty DataFrame
Columns: [EMPLOYEE_ID, SALARY, DEPARTMENT_ID]
Index: []
********************
20.0
     EMPLOYEE_ID  SALARY  DEPARTMENT_ID
101          201   13000           20.0
102          202    6000           20.0
부서평균:  9500.0
     EMPLOYEE_ID  SALARY  DEPARTMENT_ID
101          201   13000           20.0
********************
30.0
    EMPLOYEE_ID  SALARY  DEPARTMENT_ID
14          114   11000           30.0
15          115    3100           30.0
16          116    2900           30.0
17          117    2800           30.0
18          118    2600           30.0
19          119    2500           30.0
부서평균:  4150.0
    EMPLOYEE_ID  SALARY  DEPARTMENT_ID
14          114   11000           30.0
********************
40.0
     EMPLOYEE_ID  SALARY  DEPARTMENT_ID
103          203    6500           40.0
부서평균:  6500.0
Empty DataFrame
Columns: [EMPLOYEE_ID, SALARY, DEPARTMENT

,EMPLOYEE_ID,SALARY,DEPARTMENT_ID
101,201,13000,20.0
14,114,11000,30.0
20,120,8000,50.0
21,121,8200,50.0
22,122,7900,50.0
23,123,6500,50.0
24,124,5800,50.0
37,137,3600,50.0
41,141,3500,50.0
84,184,4200,50.0


### [문제109] 2003년 이전의 입사한 사원들의 정보를 출력하세요.

In [84]:
# soltuion 1 : str로 비교하기
## str.slice : Series에서 slice로 비교하는 방법

emp[emp['HIRE_DATE'].str.slice(0,4) <= '2003'] # 그래도 값을 비교할때는 date형으로 비교하는게 낫다

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
0,100,Steven,King,SKING,515.123.4567,2003-06-17,AD_PRES,24000,NaN,NaN,90.0
2,102,Lex,De Haan,LDEHAAN,515.123.4569,2001-01-13,AD_VP,17000,NaN,100.0,90.0
8,108,Nancy,Greenberg,NGREENBE,515.124.4569,2002-08-17,FI_MGR,12008,NaN,101.0,100.0
9,109,Daniel,Faviet,DFAVIET,515.124.4169,2002-08-16,FI_ACCOUNT,9000,NaN,108.0,100.0
14,114,Den,Raphaely,DRAPHEAL,515.127.4561,2002-12-07,PU_MAN,11000,NaN,100.0,30.0
15,115,Alexander,Khoo,AKHOO,515.127.4562,2003-05-18,PU_CLERK,3100,NaN,114.0,30.0
22,122,Payam,Kaufling,PKAUFLIN,650.123.3234,2003-05-01,ST_MAN,7900,NaN,100.0,50.0
37,137,Renske,Ladwig,RLADWIG,650.121.1234,2003-07-14,ST_CLERK,3600,NaN,123.0,50.0
41,141,Trenna,Rajs,TRAJS,650.121.8009,2003-10-17,ST_CLERK,3500,NaN,124.0,50.0
100,200,Jennifer,Whalen,JWHALEN,515.123.4444,2003-09-17,AD_ASST,4400,NaN,101.0,10.0


In [85]:
emp[emp['HIRE_DATE'].str.slice(0,4).astype('int') <= 2003][['EMPLOYEE_ID','HIRE_DATE']]

,EMPLOYEE_ID,HIRE_DATE
0,100,2003-06-17
2,102,2001-01-13
8,108,2002-08-17
9,109,2002-08-16
14,114,2002-12-07
15,115,2003-05-18
22,122,2003-05-01
37,137,2003-07-14
41,141,2003-10-17
100,200,2003-09-17


In [86]:
# solution 2 

emp[emp['HIRE_DATE']<='2003-01-01'][['EMPLOYEE_ID','HIRE_DATE']]

,EMPLOYEE_ID,HIRE_DATE
2,102,2001-01-13
8,108,2002-08-17
9,109,2002-08-16
14,114,2002-12-07
103,203,2002-06-07
104,204,2002-06-07
105,205,2002-06-07
106,206,2002-06-07


In [87]:
# soltuion 3 : date type으로 변환

emp[pd.to_datetime(emp['HIRE_DATE'],format = '%Y-%m-%d') <= '2003-01-01'][['EMPLOYEE_ID','HIRE_DATE']]

,EMPLOYEE_ID,HIRE_DATE
2,102,2001-01-13
8,108,2002-08-17
9,109,2002-08-16
14,114,2002-12-07
103,203,2002-06-07
104,204,2002-06-07
105,205,2002-06-07
106,206,2002-06-07


In [88]:
emp[pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d')< pd.to_datetime('2003-01-01')]

,EMPLOYEE_ID,FIRST_NAME,LAST_NAME,EMAIL,PHONE_NUMBER,HIRE_DATE,JOB_ID,SALARY,COMMISSION_PCT,MANAGER_ID,DEPARTMENT_ID
2,102,Lex,De Haan,LDEHAAN,515.123.4569,2001-01-13,AD_VP,17000,NaN,100.0,90.0
8,108,Nancy,Greenberg,NGREENBE,515.124.4569,2002-08-17,FI_MGR,12008,NaN,101.0,100.0
9,109,Daniel,Faviet,DFAVIET,515.124.4169,2002-08-16,FI_ACCOUNT,9000,NaN,108.0,100.0
14,114,Den,Raphaely,DRAPHEAL,515.127.4561,2002-12-07,PU_MAN,11000,NaN,100.0,30.0
103,203,Susan,Mavris,SMAVRIS,515.123.7777,2002-06-07,HR_REP,6500,NaN,101.0,40.0
104,204,Hermann,Baer,HBAER,515.123.8888,2002-06-07,PR_REP,10000,NaN,101.0,70.0
105,205,Shelley,Higgins,SHIGGINS,515.123.8080,2002-06-07,AC_MGR,12008,NaN,101.0,110.0
106,206,William,Gietz,WGIETZ,515.123.8181,2002-06-07,AC_ACCOUNT,8300,NaN,205.0,110.0


### [문제110] 2005년도에 입사한 사원들의 급여의 총액을 구하세요

In [89]:
# solution : str 

emp[emp['HIRE_DATE'].str.slice(0,4).astype(int) == 2005]['SALARY'].sum()

197900

In [94]:
# solution 2 : date 

emp[pd.to_datetime(emp['HIRE_DATE'],format = '%Y-%m-%d').dt.year == 2005]['SALARY'].sum()

197900

In [97]:
# soltuion 3: hire_date를 date 형식으로 수정 후 사용
emp['HIRE_DATE'] = pd.to_datetime(emp['HIRE_DATE'],format = '%Y-%m-%d')
emp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   EMPLOYEE_ID     107 non-null    int64         
 1   FIRST_NAME      107 non-null    object        
 2   LAST_NAME       107 non-null    object        
 3   EMAIL           107 non-null    object        
 4   PHONE_NUMBER    107 non-null    object        
 5   HIRE_DATE       107 non-null    datetime64[ns]
 6   JOB_ID          107 non-null    object        
 7   SALARY          107 non-null    int64         
 8   COMMISSION_PCT  35 non-null     float64       
 9   MANAGER_ID      106 non-null    float64       
 10  DEPARTMENT_ID   106 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 9.3+ KB


In [100]:
# parse_dates : csv read시 datetime으로 불러오기
## 이 테이블에 대해서 기본 정보를 알 수 있을 때, parser_dates로 불러올 수 있는데 format이 다를 수 있기 때문에 date_parser 사용해서 format 지정해주기

emp = pd.read_csv("c:/data/employees.csv", parse_dates = ['HIRE_DATE'], date_parser= lambda arg: pd.to_datetime(arg, format = '%Y-%m-%d'))

emp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   EMPLOYEE_ID     107 non-null    int64         
 1   FIRST_NAME      107 non-null    object        
 2   LAST_NAME       107 non-null    object        
 3   EMAIL           107 non-null    object        
 4   PHONE_NUMBER    107 non-null    object        
 5   HIRE_DATE       107 non-null    datetime64[ns]
 6   JOB_ID          107 non-null    object        
 7   SALARY          107 non-null    int64         
 8   COMMISSION_PCT  35 non-null     float64       
 9   MANAGER_ID      106 non-null    float64       
 10  DEPARTMENT_ID   106 non-null    float64       
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 9.3+ KB


### [문제111] 년도별 총액급여를 출력해주세요.

In [102]:
emp['SALARY'].groupby(emp['HIRE_DATE'].dt.year).sum()

HIRE_DATE
2001     17000
2002     68816
2003     46500
2004     86000
2005    197900
2006    121100
2007     94900
2008     59200
Name: SALARY, dtype: int64

### [문제112] 요일별 입사 인원수를 출력해주세요. 출력되는 화면의 요일은 한글로 출력해주세요.

In [103]:
# 입사 인원 수 뽑기

emp['EMPLOYEE_ID'].groupby(emp['HIRE_DATE'].dt.day_name()).count()

HIRE_DATE
Friday       19
Monday       10
Saturday     19
Sunday       15
Thursday     16
Tuesday      13
Wednesday    15
Name: EMPLOYEE_ID, dtype: int64

In [114]:
# weekday 이용해서 index를 월~일로 변경하기 

week = emp['EMPLOYEE_ID'].groupby(emp['HIRE_DATE'].dt.dayofweek).count()
week.sort_index(ascending = True)

HIRE_DATE
0    10
1    13
2    15
3    16
4    19
5    19
6    15
Name: EMPLOYEE_ID, dtype: int64

In [115]:
week.index = ['월','화','수','목','금','토','일']
week = week.reset_index()

week.columns = ['요일','인원수'] #편한거로 선택

week

,요일,인원수
0,월,10
1,화,13
2,수,15
3,목,16
4,금,19
5,토,19
6,일,15


### [문제113] 년도, 분기별 급여 총액을 출력해주세요.


In [123]:
## soltuion 1 : Series 다중 인덱스 

x = emp.groupby([emp['HIRE_DATE'].dt.year, emp['HIRE_DATE'].dt.quarter])['SALARY'].sum()
result = x.unstack().fillna(0) # 피봇, NaN값 채우기
result.columns = ['1분기','2분기','3분기','4분기']
result.index.name = '년도'
result

,1분기,2분기,3분기,4분기
년도,,,,
2001,17000.0,0.0,0.0,0.0
2002,0.0,36808.0,21008.0,11000.0
2003,0.0,35000.0,8000.0,3500.0
2004,40700.0,14300.0,17000.0,14000.0
2005,86900.0,16800.0,60800.0,33400.0
2006,69400.0,20400.0,14200.0,17100.0
2007,36600.0,20200.0,2500.0,35600.0
2008,46900.0,12300.0,0.0,0.0


In [126]:
# solution 2 : pd.pivot_table 

result = pd.pivot_table(data = emp,
                        index = emp['HIRE_DATE'].dt.year,
                       columns = emp['HIRE_DATE'].dt.quarter,
                       values = 'SALARY',
                       aggfunc = 'sum',
                       fill_value = 0)

result.columns = ['1분기','2분기','3분기','4분기']
result.index.name = '년도'

result 

,1분기,2분기,3분기,4분기
년도,,,,
2001,17000,0,0,0
2002,0,36808,21008,11000
2003,0,35000,8000,3500
2004,40700,14300,17000,14000
2005,86900,16800,60800,33400
2006,69400,20400,14200,17100
2007,36600,20200,2500,35600
2008,46900,12300,0,0


### [문제114] 년도, 요일별 입사 인원수를 구하세요

In [133]:
# soltuion 1 

result = emp.groupby([emp['HIRE_DATE'].dt.year,emp['HIRE_DATE'].dt.dayofweek])['DEPARTMENT_ID'].count()
result = result.unstack().fillna(0)
result.columns = ['월','화','수','목','금','토','일']
result.index.name='년도'
result = result.astype('int')
result

,월,화,수,목,금,토,일
년도,,,,,,,
2001,0,0,0,0,0,1,0
2002,0,0,0,0,5,2,0
2003,1,1,1,1,1,0,1
2004,1,3,1,1,2,0,2
2005,1,1,6,5,3,8,5
2006,1,6,2,5,2,4,4
2007,3,1,4,2,5,2,1
2008,3,1,1,1,1,2,2


In [134]:
# soltuion 2 

result = pd.pivot_table(data=emp,
               index=emp['HIRE_DATE'].dt.year,
               columns = emp['HIRE_DATE'].dt.weekday,
               values = 'EMPLOYEE_ID',
               aggfunc='count',
               fill_value=0)

result.columns = ['월','화','수','목','금','토','일']
result.index.name='년도'
result['합'] = result.sum(axis=1) # 행의 합 == 열의 합 
result.loc['합'] = result.sum(axis=0) # 열의 합 == 행의 합
result = result.astype('int')
result

,월,화,수,목,금,토,일,합
년도,,,,,,,,
2001,0,0,0,0,0,1,0,1
2002,0,0,0,0,5,2,0,7
2003,1,1,1,1,1,0,1,6
2004,1,3,1,1,2,0,2,10
2005,1,1,6,5,3,8,5,29
2006,1,6,2,5,2,4,4,24
2007,3,1,4,3,5,2,1,19
2008,3,1,1,1,1,2,2,11
합,10,13,15,16,19,19,15,107


### [문제115] last_name, 근무일수를 출력해주세요.


In [136]:
DataFrame({'이름' : emp['LAST_NAME'],
           '근무일수':(pd.Timestamp.now() - emp['HIRE_DATE']).dt.days})

,이름,근무일수
0,King,7138
1,Kochhar,6311
2,De Haan,8023
3,Hunold,6207
4,Ernst,5704
...,...,...
102,Fay,6346
103,Mavris,7513
104,Baer,7513
105,Higgins,7513


## df.to_csv(경로) 
* pandas에서 제공
* DataFrame을 CSV 파일로 떨어트릴 때 유용하다.

In [27]:
new= DataFrame() # 빈 df 만들기 가능 
df.to_csv("c:/data/new.csv",index=False)

## os.path.exists : 파일의 존재유무를 체크하는 함수 

In [29]:
import os 

os.path.exists("c:/data/yob1997.txt") # 있다면 True, 없으면 False

False

In [30]:
##  file이 존재하지 않으면 작업을 종료하고 싶다 

for i in range(2001,2017):
    file = 'c:/data/yob{}.txt'.format(i)
    if os.path.exists(file):
        temp = pd.read_csv('c:/data/yob{}.txt'.format(i),names = ['name','gender','birth'])
        x = temp['birth'].groupby(temp['gender']).sum()
        df = df.append({'년도':i,'여자':x.loc['F'], '남자':x.loc['M']},ignore_index=True) # df에 append하려면 list        
    else:
        print(file,'No file')

c:/data/yob2001.txt No file
c:/data/yob2002.txt No file
c:/data/yob2003.txt No file
c:/data/yob2004.txt No file
c:/data/yob2005.txt No file
c:/data/yob2006.txt No file
c:/data/yob2007.txt No file
c:/data/yob2008.txt No file
c:/data/yob2009.txt No file
c:/data/yob2010.txt No file
c:/data/yob2011.txt No file
c:/data/yob2012.txt No file


C:\Users\82108\AppData\Local\Temp\ipykernel_13564\3723049611.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'년도':i,'여자':x.loc['F'], '남자':x.loc['M']},ignore_index=True) # df에 append하려면 list
C:\Users\82108\AppData\Local\Temp\ipykernel_13564\3723049611.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'년도':i,'여자':x.loc['F'], '남자':x.loc['M']},ignore_index=True) # df에 append하려면 list
C:\Users\82108\AppData\Local\Temp\ipykernel_13564\3723049611.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'년도':i,'여자':x.loc['F'], '남자':x.loc['M']},ignore_index=True) # df에 append하려면 list
C:\Users\82108\AppData\Local\Temp\ipykernel_13564\3723049611.py:8: FutureWarning: The frame.append meth

## 판다스의 날짜 함수들

* datetime 
* Timestamp 
* Timedelta
* x.dt.year/month...

In [50]:
import pandas as pd 
import datetime 

In [51]:
datetime.datetime.now()

datetime.datetime(2023, 1, 1, 14, 34, 44, 351480)

In [52]:
pd.datetime.now()

C:\Users\82108\AppData\Local\Temp\ipykernel_13564\627754903.py:1: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  pd.datetime.now()


datetime.datetime(2023, 1, 1, 14, 34, 51, 612891)

In [53]:
pd.Timestamp.today()

Timestamp('2023-01-01 14:35:23.771476')

In [54]:
pd.Timestamp.now()

Timestamp('2023-01-01 14:35:33.092428')

### 사용자 정의 시간 : 날짜를 고정해야 할 때 사용됨

In [55]:
pd.Timestamp(year = 2021, month = 9, day = 28, hour= 14, minute = 5, second= 10, microsecond=1,tz='Asia/Seoul')

Timestamp('2021-09-28 14:05:10.000001+0900', tz='Asia/Seoul')

### pd.to_datetime : char -> timestasmp 형 변환 함수 
* 4.1.과 같이 하나하나 지정하기 힘들 때 심플하게 사용 가능 

In [57]:
pd.to_datetime('2021-09-28')
pd.to_datetime('2021-9-28')
pd.to_datetime('2021/09/28')
pd.to_datetime('20210928')

Timestamp('2021-09-28 00:00:00')

In [59]:
pd.to_datetime('09282021') # format이 맞지 않아 error 

ParserError: month must be in 1..12: 09282021

In [60]:
pd.to_datetime('09282021',format='%m%d%Y') # format 사용하면 됨

Timestamp('2021-09-28 00:00:00')

In [61]:
pd.to_datetime('20210928141000',format='%Y%m%d%H%M%S') 

Timestamp('2021-09-28 14:10:00')

In [62]:
day = ['2021-09-27','2021-09-28']
day

['2021-09-27', '2021-09-28']

In [64]:
# pd.to_datetime(Series(day)).year # Series라서 사용이 안됨 
pd.to_datetime(day).year # list는 사용이 가능

Int64Index([2021, 2021], dtype='int64')

### 특정한 날짜 추출하기 

* year 
* month 
* day
* hour...

In [65]:
pd.Timestamp.now()
pd.Timestamp.now().year
pd.Timestamp.now().month
pd.Timestamp.now().day
pd.Timestamp.now().hour
pd.Timestamp.now().minute
pd.Timestamp.now().second
pd.Timestamp.now().microsecond
pd.Timestamp.now().dayofweek #0~6, 월~일
pd.Timestamp.now().day_name()
pd.Timestamp.now().quarter

1

In [68]:
pd.Timestamp.now() - pd.to_datetime('2021-12-3')
(pd.to_datetime('2021-12-3')-pd.Timestamp.now()).days

-395

### Timedelta(x) : x 만큼의 날짜를 연산한다.
* year 연산이 불가능하다

In [71]:
pd.Timestamp.now() + pd.Timedelta('100 days')
pd.Timestamp.now() - pd.Timedelta('100 days') 

Timestamp('2022-09-23 17:07:13.766867')

In [72]:
pd.Timestamp.now() + pd.Timedelta('4 hours')

Timestamp('2023-01-01 21:07:30.972053')

In [73]:
pd.Timestamp.now() + pd.Timedelta('60 minutes') 
pd.Timestamp.now() + pd.Timedelta('59 min') 

Timestamp('2023-01-01 18:06:50.734727')

In [74]:
pd.Timestamp.now() + pd.Timedelta('3600 second') 
pd.Timestamp.now() + pd.Timedelta('3600 sec') 

Timestamp('2023-01-01 18:07:56.240919')

In [75]:
# 복잡한 연산도 가능하다 
pd.Timestamp.now() + pd.Timedelta('1 days 10 hours') 
pd.Timestamp.now() + pd.Timedelta('4:10:00') 

Timestamp('2023-01-01 21:18:07.207585')

In [76]:
pd.Timestamp.now() + pd.Timedelta('1 years') # error

ValueError: invalid unit abbreviation: years

### pd.offsets.DateOffset() : 날짜를 더함. year도 가능.
* 단위를 지정해서 연산이 가능하다(단위를 지정하지 않으면 에러가 뜸) 

In [77]:
pd.Timestamp.now() + pd.offsets.DateOffset(years = 1)

Timestamp('2024-01-01 17:09:14.664417')

In [79]:
# 지정해서 연산이 가능하다
pd.Timestamp.now() + pd.DateOffset(year=2002,month=1,day=1,hour=0,minute=0)

Timestamp('2002-01-01 00:00:28.757504')

In [82]:
# 단위를 지정하지 않으면 연산이 불가능하다 
pd.Timestamp.now() + pd.DateOffset('4:10:00')

TypeError: `n` argument must be an integer, got <class 'str'>

### x.dt.year/month/day... : 특정 날짜를 뽑아낼 수 있는 함수 

In [96]:
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.year
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.month
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.day
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.hour
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.minute
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.second
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.microsecond
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.dayofweek
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.weekday
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.day_name()
pd.to_datetime(emp['HIRE_DATE'],format='%Y-%m-%d').dt.quarter

0      2
1      3
2      1
3      1
4      2
      ..
102    3
103    2
104    2
105    2
106    2
Name: HIRE_DATE, Length: 107, dtype: int64